# Advanced PINN Techniques

This notebook explores advanced techniques for improving PINN performance and handling more complex scenarios.

## Topics Covered
1. Adaptive loss weighting
2. Advanced network architectures
3. Domain decomposition
4. Transfer learning for PDEs
5. Inverse problems
6. Multi-physics coupling

In [ ]:
# Setup
import sys
sys.path.append('..')

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from src.model import WavePINN
from src.losses import PhysicsInformedLoss
from src.train import train_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.dpi'] = 100

## 1. Adaptive Loss Weighting

One of the biggest challenges in PINNs is balancing different loss components. Adaptive weighting adjusts weights during training.

In [ ]:
class AdaptiveWeightedLoss:
    """Adaptive loss weighting based on gradient magnitudes"""
    
    def __init__(self, initial_weights=None):
        if initial_weights is None:
            initial_weights = {'pde': 1.0, 'ic': 1.0, 'bc': 1.0}
        
        self.weights = initial_weights.copy()
        self.grad_history = {key: [] for key in self.weights}
        self.update_freq = 100
        self.iteration = 0
    
    def compute_gradients(self, losses, model):
        """Compute gradient magnitudes for each loss component"""
        grads = {}
        
        for key, loss in losses.items():
            if key != 'total':
                # Compute gradients
                grad = torch.autograd.grad(loss, model.parameters(), 
                                         retain_graph=True, allow_unused=True)
                
                # Compute gradient magnitude
                grad_norm = 0
                for g in grad:
                    if g is not None:
                        grad_norm += g.norm().item()**2
                
                grads[key] = np.sqrt(grad_norm)
        
        return grads
    
    def update_weights(self, losses, model):
        """Update weights based on gradient balancing"""
        self.iteration += 1
        
        # Compute gradients
        grads = self.compute_gradients(losses, model)
        
        # Store history
        for key in grads:
            self.grad_history[key].append(grads[key])
        
        # Update weights periodically
        if self.iteration % self.update_freq == 0 and self.iteration > 0:
            # Compute mean gradients over recent history
            mean_grads = {}
            for key in self.weights:
                if key in self.grad_history and len(self.grad_history[key]) > 0:
                    mean_grads[key] = np.mean(self.grad_history[key][-50:])
            
            # Balance gradients
            if len(mean_grads) > 0:
                max_grad = max(mean_grads.values())
                for key in self.weights:
                    if key in mean_grads and mean_grads[key] > 0:
                        self.weights[key] *= max_grad / mean_grads[key]
                
                # Normalize weights
                total_weight = sum(self.weights.values())
                for key in self.weights:
                    self.weights[key] /= total_weight
        
        return self.weights

In [ ]:
# Demonstrate adaptive weighting
def train_with_adaptive_weights(model, epochs=2000):
    """Train PINN with adaptive loss weighting"""
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    adaptive_loss = AdaptiveWeightedLoss()
    
    history = {'loss': [], 'weights_pde': [], 'weights_ic': [], 'weights_bc': []}
    
    for epoch in tqdm(range(epochs), desc='Training with adaptive weights'):
        # Generate training data
        x_pde = torch.rand(1000, 1, device=device, requires_grad=True)
        t_pde = torch.rand(1000, 1, device=device, requires_grad=True)
        x_ic = torch.rand(200, 1, device=device)
        x_bc = torch.cat([torch.zeros(100, 1), torch.ones(100, 1)], dim=0).to(device)
        t_bc = torch.rand(200, 1, device=device)
        
        # Compute individual losses
        losses = {}
        
        # PDE loss
        u = model(x_pde, t_pde)
        u_x = torch.autograd.grad(u, x_pde, torch.ones_like(u), create_graph=True)[0]
        u_t = torch.autograd.grad(u, t_pde, torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x_pde, torch.ones_like(u_x), create_graph=True)[0]
        u_tt = torch.autograd.grad(u_t, t_pde, torch.ones_like(u_t), create_graph=True)[0]
        losses['pde'] = torch.mean((u_tt - u_xx)**2)
        
        # IC loss
        t_ic = torch.zeros_like(x_ic)
        u_ic = model(x_ic, t_ic)
        losses['ic'] = torch.mean((u_ic - torch.sin(np.pi * x_ic))**2)
        
        # BC loss
        u_bc = model(x_bc, t_bc)
        losses['bc'] = torch.mean(u_bc**2)
        
        # Update weights
        weights = adaptive_loss.update_weights(losses, model)
        
        # Total loss
        total_loss = sum(weights[key] * losses[key] for key in losses)
        
        # Backward pass
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        # Record history
        history['loss'].append(total_loss.item())
        history['weights_pde'].append(weights['pde'])
        history['weights_ic'].append(weights['ic'])
        history['weights_bc'].append(weights['bc'])
    
    return history

# Train model with adaptive weights
model_adaptive = WavePINN().to(device)
history_adaptive = train_with_adaptive_weights(model_adaptive)

# Visualize weight evolution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Loss history
ax1.semilogy(history_adaptive['loss'])
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Total Loss')
ax1.set_title('Training Loss with Adaptive Weighting')
ax1.grid(True)

# Weight evolution
ax2.plot(history_adaptive['weights_pde'], label='PDE weight')
ax2.plot(history_adaptive['weights_ic'], label='IC weight')
ax2.plot(history_adaptive['weights_bc'], label='BC weight')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Weight Value')
ax2.set_title('Adaptive Weight Evolution')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## 2. Advanced Network Architectures

Different architectures can significantly impact PINN performance. Let's explore some advanced options:

In [ ]:
class ResidualPINN(nn.Module):
    """PINN with residual connections"""
    
    def __init__(self, input_dim=2, hidden_dim=64, num_blocks=3):
        super(ResidualPINN, self).__init__()
        
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        
        # Residual blocks
        self.blocks = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.Tanh(),
                nn.Linear(hidden_dim, hidden_dim)
            ) for _ in range(num_blocks)
        ])
        
        self.output_layer = nn.Linear(hidden_dim, 1)
        self.activation = nn.Tanh()
    
    def forward(self, x, t):
        # Input
        inputs = torch.cat([x, t], dim=1)
        out = self.activation(self.input_layer(inputs))
        
        # Residual blocks
        for block in self.blocks:
            residual = out
            out = block(out)
            out = self.activation(out + residual)  # Residual connection
        
        # Output
        out = self.output_layer(out)
        return out


class FourierFeaturePINN(nn.Module):
    """PINN with Fourier feature encoding"""
    
    def __init__(self, input_dim=2, hidden_dim=64, num_frequencies=10):
        super(FourierFeaturePINN, self).__init__()
        
        # Fourier feature frequencies
        self.frequencies = 2**torch.linspace(0, num_frequencies-1, num_frequencies)
        self.frequencies = self.frequencies.reshape(1, -1)
        
        # Network
        feature_dim = input_dim * num_frequencies * 2
        self.net = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
    
    def fourier_features(self, x):
        """Compute Fourier features"""
        self.frequencies = self.frequencies.to(x.device)
        x_proj = x @ self.frequencies
        return torch.cat([torch.sin(2 * np.pi * x_proj), 
                         torch.cos(2 * np.pi * x_proj)], dim=-1)
    
    def forward(self, x, t):
        # Compute Fourier features
        inputs = torch.cat([x, t], dim=1)
        features = self.fourier_features(inputs)
        features = features.reshape(features.shape[0], -1)
        
        return self.net(features)


# Compare architectures
architectures = {
    'Standard': WavePINN(),
    'Residual': ResidualPINN(),
    'Fourier': FourierFeaturePINN()
}

# Count parameters
for name, model in architectures.items():
    params = sum(p.numel() for p in model.parameters())
    print(f"{name} PINN: {params:,} parameters")

## 3. Domain Decomposition

For complex domains or large-scale problems, domain decomposition can improve training efficiency:

In [ ]:
class DomainDecompositionPINN:
    """PINN with domain decomposition"""
    
    def __init__(self, num_subdomains=4):
        self.num_subdomains = num_subdomains
        self.models = []
        self.boundaries = []
        
        # Create subdomains
        for i in range(num_subdomains):
            self.models.append(WavePINN().to(device))
            
            # Define subdomain boundaries
            x_min = i / num_subdomains
            x_max = (i + 1) / num_subdomains
            self.boundaries.append((x_min, x_max))
    
    def forward(self, x, t):
        """Evaluate the appropriate subdomain model"""
        outputs = torch.zeros_like(x)
        
        for i, (model, (x_min, x_max)) in enumerate(zip(self.models, self.boundaries)):
            # Find points in this subdomain
            mask = (x >= x_min) & (x <= x_max)
            
            if mask.any():
                x_sub = x[mask]
                t_sub = t[mask]
                
                # Transform to local coordinates
                x_local = (x_sub - x_min) / (x_max - x_min)
                
                # Evaluate model
                outputs[mask] = model(x_local, t_sub)
        
        return outputs
    
    def train_step(self, optimizer, loss_fn):
        """Train all subdomain models with interface conditions"""
        total_loss = 0
        
        for i, (model, (x_min, x_max)) in enumerate(zip(self.models, self.boundaries)):
            # Generate training data for subdomain
            x_pde = torch.rand(250, 1, device=device) * (x_max - x_min) + x_min
            t_pde = torch.rand(250, 1, device=device)
            
            # Subdomain loss
            loss = loss_fn(model, x_pde, t_pde)
            
            # Interface conditions (continuity)
            if i < self.num_subdomains - 1:
                x_interface = torch.full((50, 1), x_max, device=device)
                t_interface = torch.rand(50, 1, device=device)
                
                # Values from current and next subdomain
                u_current = model(torch.ones_like(x_interface), t_interface)
                u_next = self.models[i+1](torch.zeros_like(x_interface), t_interface)
                
                # Continuity loss
                interface_loss = torch.mean((u_current - u_next)**2)
                loss += 10.0 * interface_loss
            
            total_loss += loss
        
        # Backward pass
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        return total_loss.item()

# Visualize domain decomposition
fig, ax = plt.subplots(figsize=(10, 6))

# Create domain decomposition
dd_pinn = DomainDecompositionPINN(num_subdomains=4)

# Plot subdomains
colors = plt.cm.viridis(np.linspace(0, 1, dd_pinn.num_subdomains))
for i, ((x_min, x_max), color) in enumerate(zip(dd_pinn.boundaries, colors)):
    ax.add_patch(plt.Rectangle((x_min, 0), x_max-x_min, 1, 
                              facecolor=color, alpha=0.3, edgecolor='black', linewidth=2))
    ax.text((x_min + x_max)/2, 0.5, f'Domain {i+1}', 
            ha='center', va='center', fontsize=12, weight='bold')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('t', fontsize=12)
ax.set_title('Domain Decomposition for PINN', fontsize=14)
ax.set_aspect('equal')

# Add interface lines
for i in range(1, dd_pinn.num_subdomains):
    x_interface = i / dd_pinn.num_subdomains
    ax.axvline(x=x_interface, color='red', linestyle='--', linewidth=2, alpha=0.7)
    ax.text(x_interface, 1.05, 'Interface', ha='center', color='red', fontsize=10)

plt.tight_layout()
plt.show()

## 4. Transfer Learning for PDEs

We can use a pre-trained PINN as a starting point for solving related problems:

In [ ]:
def transfer_learning_demo():
    """Demonstrate transfer learning for PDEs"""
    
    # Step 1: Train base model on standard wave equation (c=1)
    print("Step 1: Training base model (c=1)...")
    base_model = WavePINN().to(device)
    history_base = train_model(base_model, epochs=2000, verbose=False)
    
    # Step 2: Use as initialization for different wave speed (c=2)
    print("\nStep 2: Transfer learning for c=2...")
    
    # Create two models: one with transfer, one without
    model_transfer = WavePINN().to(device)
    model_transfer.load_state_dict(base_model.state_dict())  # Transfer weights
    
    model_scratch = WavePINN().to(device)  # Train from scratch
    
    # Modified loss for c=2
    from src.losses import PhysicsInformedLoss
    loss_fn_c2 = PhysicsInformedLoss(wave_speed=2.0)
    
    # Train both models
    history_transfer = []
    history_scratch = []
    
    optimizer_transfer = torch.optim.Adam(model_transfer.parameters(), lr=1e-3)
    optimizer_scratch = torch.optim.Adam(model_scratch.parameters(), lr=1e-3)
    
    for epoch in tqdm(range(1000), desc='Training'):
        # Generate data
        x_pde = torch.rand(1000, 1, device=device)
        t_pde = torch.rand(1000, 1, device=device)
        x_ic = torch.rand(200, 1, device=device)
        x_bc = torch.cat([torch.zeros(100, 1), torch.ones(100, 1)], dim=0).to(device)
        t_bc = torch.rand(200, 1, device=device)
        
        # Train transfer model
        loss_transfer, _ = loss_fn_c2(model_transfer, x_pde, t_pde, x_ic, x_bc, t_bc)
        optimizer_transfer.zero_grad()
        loss_transfer.backward()
        optimizer_transfer.step()
        history_transfer.append(loss_transfer.item())
        
        # Train scratch model
        loss_scratch, _ = loss_fn_c2(model_scratch, x_pde, t_pde, x_ic, x_bc, t_bc)
        optimizer_scratch.zero_grad()
        loss_scratch.backward()
        optimizer_scratch.step()
        history_scratch.append(loss_scratch.item())
    
    return history_transfer, history_scratch

# Run transfer learning experiment
history_transfer, history_scratch = transfer_learning_demo()

# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Training curves
ax1.semilogy(history_transfer, 'b-', linewidth=2, label='With Transfer Learning')
ax1.semilogy(history_scratch, 'r--', linewidth=2, label='From Scratch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Transfer Learning Comparison')
ax1.legend()
ax1.grid(True)

# Convergence speed
threshold = 0.01
epochs_transfer = next((i for i, loss in enumerate(history_transfer) if loss < threshold), len(history_transfer))
epochs_scratch = next((i for i, loss in enumerate(history_scratch) if loss < threshold), len(history_scratch))

ax2.bar(['Transfer Learning', 'From Scratch'], [epochs_transfer, epochs_scratch], 
        color=['blue', 'red'], alpha=0.7)
ax2.set_ylabel('Epochs to Reach Loss < 0.01')
ax2.set_title('Convergence Speed')

# Add text
speedup = epochs_scratch / epochs_transfer if epochs_transfer > 0 else float('inf')
ax2.text(0.5, max(epochs_transfer, epochs_scratch) * 0.5, 
         f'{speedup:.1f}x speedup', ha='center', fontsize=14, weight='bold')

plt.tight_layout()
plt.show()

## 5. Inverse Problems

PINNs can solve inverse problems - inferring unknown parameters from observations:

In [ ]:
class InversePINN(nn.Module):
    """PINN for inverse problems - learns both solution and parameters"""
    
    def __init__(self):
        super(InversePINN, self).__init__()
        
        # Solution network
        self.solution_net = WavePINN()
        
        # Unknown parameter (wave speed)
        self.log_c = nn.Parameter(torch.tensor([0.0]))  # log(c) for positivity
    
    def forward(self, x, t):
        return self.solution_net(x, t)
    
    @property
    def wave_speed(self):
        return torch.exp(self.log_c)

def solve_inverse_problem():
    """Solve inverse problem: find wave speed from observations"""
    
    # True wave speed (unknown to the model)
    true_c = 1.5
    
    # Generate synthetic observations
    n_obs = 100
    x_obs = torch.rand(n_obs, 1) * 0.8 + 0.1  # Avoid boundaries
    t_obs = torch.rand(n_obs, 1) * 0.8 + 0.1
    
    # True solution
    u_obs = torch.sin(np.pi * x_obs) * torch.cos(np.pi * true_c * t_obs)
    u_obs += 0.01 * torch.randn_like(u_obs)  # Add noise
    
    # Create inverse PINN
    model = InversePINN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    # Training
    history_c = []
    history_loss = []
    
    for epoch in tqdm(range(3000), desc='Solving inverse problem'):
        # PDE collocation points
        x_pde = torch.rand(500, 1, device=device, requires_grad=True)
        t_pde = torch.rand(500, 1, device=device, requires_grad=True)
        
        # Forward pass
        u = model(x_pde, t_pde)
        
        # Compute derivatives
        u_x = torch.autograd.grad(u, x_pde, torch.ones_like(u), create_graph=True)[0]
        u_t = torch.autograd.grad(u, t_pde, torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x_pde, torch.ones_like(u_x), create_graph=True)[0]
        u_tt = torch.autograd.grad(u_t, t_pde, torch.ones_like(u_t), create_graph=True)[0]
        
        # PDE residual with unknown parameter
        c_squared = model.wave_speed**2
        pde_loss = torch.mean((u_tt - c_squared * u_xx)**2)
        
        # Data loss
        x_obs_gpu = x_obs.to(device)
        t_obs_gpu = t_obs.to(device)
        u_obs_gpu = u_obs.to(device)
        
        u_pred = model(x_obs_gpu, t_obs_gpu)
        data_loss = torch.mean((u_pred - u_obs_gpu)**2)
        
        # Initial condition
        x_ic = torch.rand(100, 1, device=device)
        t_ic = torch.zeros_like(x_ic)
        u_ic = model(x_ic, t_ic)
        ic_loss = torch.mean((u_ic - torch.sin(np.pi * x_ic))**2)
        
        # Total loss
        loss = pde_loss + 100 * data_loss + 10 * ic_loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Record history
        history_c.append(model.wave_speed.item())
        history_loss.append(loss.item())
    
    return model, history_c, history_loss, true_c

# Solve inverse problem
model_inverse, history_c, history_loss, true_c = solve_inverse_problem()

# Visualize results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# Loss history
ax1.semilogy(history_loss)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Total Loss')
ax1.set_title('Training Loss')
ax1.grid(True)

# Parameter estimation
ax2.plot(history_c, 'b-', linewidth=2)
ax2.axhline(y=true_c, color='r', linestyle='--', linewidth=2, label=f'True c = {true_c}')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Estimated Wave Speed c')
ax2.set_title('Parameter Estimation')
ax2.legend()
ax2.grid(True)

# Final estimate
final_c = history_c[-1]
error = abs(final_c - true_c) / true_c * 100

ax3.bar(['True', 'Estimated'], [true_c, final_c], color=['red', 'blue'], alpha=0.7)
ax3.set_ylabel('Wave Speed c')
ax3.set_title(f'Final Result (Error: {error:.2f}%)')
ax3.set_ylim(0, 2)

plt.tight_layout()
plt.show()

print(f"\nTrue wave speed: {true_c}")
print(f"Estimated wave speed: {final_c:.4f}")
print(f"Relative error: {error:.2f}%")

## 6. Multi-Physics Coupling

PINNs can handle coupled PDEs, making them suitable for multi-physics problems:

In [ ]:
class CoupledPINN(nn.Module):
    """PINN for coupled wave-heat equations"""
    
    def __init__(self):
        super(CoupledPINN, self).__init__()
        
        # Shared encoder
        self.encoder = nn.Sequential(
            nn.Linear(2, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh()
        )
        
        # Separate decoders for each field
        self.wave_decoder = nn.Sequential(
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )
        
        self.heat_decoder = nn.Sequential(
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x, t):
        inputs = torch.cat([x, t], dim=1)
        features = self.encoder(inputs)
        
        u_wave = self.wave_decoder(features)
        u_heat = self.heat_decoder(features)
        
        return u_wave, u_heat

# Visualize multi-physics coupling
fig, ax = plt.subplots(figsize=(10, 6))

# Draw coupling diagram
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

# Wave equation box
wave_box = FancyBboxPatch((0.1, 0.6), 0.3, 0.3, 
                          boxstyle="round,pad=0.05",
                          facecolor='lightblue', edgecolor='black', linewidth=2)
ax.add_patch(wave_box)
ax.text(0.25, 0.75, 'Wave Equation\n∂²u/∂t² = c²∇²u', 
        ha='center', va='center', fontsize=12, weight='bold')

# Heat equation box
heat_box = FancyBboxPatch((0.6, 0.6), 0.3, 0.3,
                          boxstyle="round,pad=0.05",
                          facecolor='lightcoral', edgecolor='black', linewidth=2)
ax.add_patch(heat_box)
ax.text(0.75, 0.75, 'Heat Equation\n∂T/∂t = α∇²T', 
        ha='center', va='center', fontsize=12, weight='bold')

# Coupling arrows
arrow1 = FancyArrowPatch((0.4, 0.75), (0.6, 0.75),
                         connectionstyle="arc3,rad=.2",
                         arrowstyle="->", mutation_scale=20,
                         linewidth=2, color='green')
ax.add_patch(arrow1)

arrow2 = FancyArrowPatch((0.6, 0.75), (0.4, 0.75),
                         connectionstyle="arc3,rad=-.2",
                         arrowstyle="->", mutation_scale=20,
                         linewidth=2, color='green')
ax.add_patch(arrow2)

# Coupling term
ax.text(0.5, 0.85, 'Coupling: u affects α', ha='center', fontsize=10, color='green')
ax.text(0.5, 0.65, 'Coupling: T affects c', ha='center', fontsize=10, color='green')

# PINN box
pinn_box = FancyBboxPatch((0.2, 0.1), 0.6, 0.3,
                          boxstyle="round,pad=0.05",
                          facecolor='lightyellow', edgecolor='black', linewidth=2)
ax.add_patch(pinn_box)
ax.text(0.5, 0.25, 'Coupled PINN\n(Shared representation)', 
        ha='center', va='center', fontsize=14, weight='bold')

# Connections to PINN
ax.arrow(0.25, 0.6, 0, -0.15, head_width=0.03, head_length=0.02, fc='black', ec='black')
ax.arrow(0.75, 0.6, 0, -0.15, head_width=0.03, head_length=0.02, fc='black', ec='black')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('Multi-Physics Coupling with PINNs', fontsize=16, weight='bold', pad=20)

plt.tight_layout()
plt.show()

## Summary

In this notebook, we explored advanced PINN techniques:

1. **Adaptive Loss Weighting**: Automatically balances different loss components during training
2. **Advanced Architectures**: Residual connections and Fourier features can improve performance
3. **Domain Decomposition**: Enables solving large-scale problems by dividing the domain
4. **Transfer Learning**: Pre-trained models can accelerate training for related problems
5. **Inverse Problems**: PINNs can infer unknown parameters from observations
6. **Multi-Physics**: Coupled PDEs can be solved with shared representations

### Key Takeaways

- **Flexibility**: PINNs can be adapted to various problem types and scales
- **Efficiency**: Advanced techniques can significantly improve training speed and accuracy
- **Versatility**: From forward to inverse problems, PINNs handle diverse scenarios
- **Extensibility**: The framework naturally extends to coupled and multi-physics problems

### Next Steps

- Explore these techniques on your own PDE problems
- Combine multiple techniques for even better performance
- Consider hardware acceleration (multi-GPU training)
- Investigate recent research papers for cutting-edge methods